In [ ]:
import sys
sys.path.append('..')
import wandb
import os

os.makedirs('../experiments', exist_ok=True)
os.makedirs('../results', exist_ok=True)

from src.dataset import get_dataloader
from src.model import get_model, count_parameters
from src.train import train

print("All imports successful")

In [ ]:
config = {
    'model': 'resnet50',
    'epochs': 5,           # start with 5 to verify everything works
    'batch_size': 32,
    'lr': 1e-4,
    'weight_decay': 1e-5,
    'pretrained': True
}

wandb.init(
    project='ai-image-detector',
    config=config,
    name='resnet50-baseline-run1'
)

In [ ]:
train_loader = get_dataloader('../data/raw/faces', split='train', batch_size=config['batch_size'])
val_loader   = get_dataloader('../data/raw/faces', split='valid', batch_size=config['batch_size'])

model = get_model(config['model'], pretrained=config['pretrained'])
count_parameters(model)

In [ ]:
# Quick sanity check — run just 1 batch to verify no errors
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
images, labels = next(iter(train_loader))
images = images.to(device)
labels = labels.float().unsqueeze(1).to(device)
outputs = model(images)
print(f'Forward pass successful. Output shape: {outputs.shape}')

In [ ]:
model = train(model, train_loader, val_loader, config)
wandb.finish()
print("Training complete")